In [1]:
from utils import sailPreprocess, loadOTSModel

In [2]:
loc = sailPreprocess()
model, tokenizer = loadOTSModel(name='t5-small', cache_dir=loc)

In [3]:
model.encoder.block[0]

T5Block(
  (layer): ModuleList(
    (0): T5LayerSelfAttention(
      (SelfAttention): T5Attention(
        (q): Linear(in_features=512, out_features=512, bias=False)
        (k): Linear(in_features=512, out_features=512, bias=False)
        (v): Linear(in_features=512, out_features=512, bias=False)
        (o): Linear(in_features=512, out_features=512, bias=False)
        (relative_attention_bias): Embedding(32, 8)
      )
      (layer_norm): T5LayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (1): T5LayerFF(
      (DenseReluDense): T5DenseReluDense(
        (wi): Linear(in_features=512, out_features=2048, bias=False)
        (wo): Linear(in_features=2048, out_features=512, bias=False)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (layer_norm): T5LayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
)

In [8]:
hasattr(model, 'transformer')

False

In [10]:
model.encoder.block[-2:]

<generator object Module.parameters at 0x7f4f3c0a7820>

In [4]:

input_ids = tokenizer('The <extra_id_0> walks in <extra_id_1> park', return_tensors='pt').input_ids
labels = tokenizer('<extra_id_0> cute dog <extra_id_1> the <extra_id_2>', return_tensors='pt').input_ids
outputs = model(input_ids=input_ids, labels=labels)
loss = outputs.loss
logits = outputs.logits

In [10]:
from torch import tensor
import torch
import numpy as np

In [189]:
lm_data = [
    tensor([[ 8555,  2375, 31238, 21249,  1949,    19,    46, 19143,   771,  8498,
          1886,     3,   390,    16, 32099,     3,     6,  7449,     5,     1]]), 
    tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
]

edit_data = [
    tensor([[   37,  1784,    13,  7449,    19, 32099,     3,     5,     1]]), 
    tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])
]

obj = [tensor([[32099, 31238, 32098,     1]]), tensor([[1, 1, 1, 1]])]
edit_obj = [tensor([[32099, 31238, 32098,     1]]), tensor([[1, 1, 1, 1]])]

In [182]:
torch.where(edit_obj[0][0,:] == 32099)

(tensor([0]),)

In [183]:
test = edit_obj[0]

In [185]:
idx = np.setxor1d(list(range(test.shape[1])), find(test, special_obj)[1])
gold_tokens = test[:,idx]

In [186]:
edit_start = utils.locateSubset(edit_data[0], torch.tensor([32099]))
edit_locs = torch.tensor([edit_start + i for i in range(gold_tokens.flatten().size()[0])])

In [187]:
gold_tokens

tensor([[31238]])

In [188]:
edit_locs

tensor([5])

In [193]:
lm_tokens, lm_mask = lm_data


lm_labels, _ = obj


edit_tokens_batch, edit_mask_batch = tuple(zip(*[edit_data]))


In [216]:
edit_labels_batch, edit_lab_mask_batch = tuple(zip(*[edit_obj]))

In [211]:
edit_tokens_batch

(tensor([[   37,  1784,    13,  7449,    19, 32099,     3,     5,     1]]),)

In [208]:
find = lambda tensor, v: torch.where(tensor[..., None] == v)
special_obj  = torch.tensor([32099, 32098, 1])

In [209]:
label_tokens, label_mask = edit_obj

In [210]:
edit_tokens[edit_mask == 1]

tensor([32099, 31238, 32098,     1])

In [205]:
edit_tokens = edit_tokens_batch[0]
edit_mask = edit_mask_batch[0]

edit_tokens = edit_tokens.squeeze(0)
indices = edit_tokens != tokenizer.pad_token_id
edit_tokens = edit_tokens[indices].unsqueeze(0)
edit_mask = edit_mask.squeeze(0)
edit_mask = edit_mask[indices].unsqueeze(0)

label_indices = edit_tokens != tokenizer.pad_token_id

In [207]:
edit_mask

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [203]:
edit_tokens


In [204]:
indices

tensor([True, True, True, True, True, True, True, True, True])

In [ ]:
def _process_edit_tokens(edit_tokens, edit_mask):
    edit_tokens = edit_tokens.squeeze(0)
    indices = edit_tokens != self.tokenizer.pad_token_id
    edit_tokens = edit_tokens[indices].unsqueeze(0)
    edit_mask = edit_mask.squeeze(0)
    edit_mask = edit_mask[indices].unsqueeze(0)

    edit_labels = torch.zeros(edit_tokens.shape, dtype=torch.long) - 100
    edit_loc = edit_tokens.shape[-1] - 5 - 1  # minus 1 for newline token
    edit_locs = torch.tensor([edit_loc + i for i in range(5)])
    edit_labels[:, edit_locs] = edit_tokens[:, edit_locs]
    gold_tokens = edit_tokens[:, edit_locs]

    edit_labels = edit_labels.to(self.device)
    edit_tokens, edit_mask = edit_tokens.to(self.device), edit_mask.to(self.device)

    gold_tokens = gold_tokens.cpu()

    return edit_tokens, edit_mask, edit_labels, edit_locs, gold_tokens

In [6]:
import datasets
from datasets import load_dataset
import utils

In [7]:
loc = utils.sailPreprocess()

In [25]:
dataset

Dataset({
    features: ['uuid', 'obj_uri', 'obj_label', 'sub_uri', 'sub_label', 'predicate_id', 'sub_surface', 'obj_surface', 'masked_sentence', 'template', 'template_negated', 'label', 'description', 'type'],
    num_rows: 65220
})

In [19]:
import imp
import masked_lm.data as mld

imp.reload(mld)

<module 'masked_lm.data' from '/juice/scr/spencerb/editable_nlp/src/masked_lm/data.py'>

In [20]:
dl = mld.MaskedLMDataloader(
    'lama', loc, 'editable',
    pct=50,
    shuffle=True
)

Edited LAMA not found ('/jagupard11/scr2/spencerb/hf/lama_edited.pkl'). Generate? (y/n) y
Generating...


Reusing dataset lama (/jagupard11/scr2/spencerb/hf/lama/trex/1.1.0/97deffae13eca0a18e77dfb3960bb31741e973586f5c1fe1ec0d6b5eece7bddc)
Loading cached shuffled indices for dataset at /jagupard11/scr2/spencerb/hf/lama/trex/1.1.0/97deffae13eca0a18e77dfb3960bb31741e973586f5c1fe1ec0d6b5eece7bddc/cache-0d278a8097866daf.arrow
100%|██████████| 1304391/1304391 [00:01<00:00, 993679.12it/s] 


Edited LAMA written to "/jagupard11/scr2/spencerb/hf/lama_edited.pkl"
self.dataset populated


In [22]:
for (a,b,c,d) in dl.train:
    print(a)
    print(b)
    print(c)
    print(d)
    
    break

Exception ignored in: Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f715f185670><function _MultiProcessingDataLoaderIter.__del__ at 0x7f715f185670>

Traceback (most recent call last):
  File "/u/nlp/anaconda/main/anaconda3/envs/editable/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1324, in __del__
Traceback (most recent call last):
  File "/u/nlp/anaconda/main/anaconda3/envs/editable/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1324, in __del__
    self._shutdown_workers()    
self._shutdown_workers()  File "/u/nlp/anaconda/main/anaconda3/envs/editable/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1316, in _shutdown_workers

  File "/u/nlp/anaconda/main/anaconda3/envs/editable/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1316, in _shutdown_workers
        if w.is_alive():
  File "/u/nlp/anaconda/main/anaconda3/envs/editable/lib/python3.8/multiprocessing/process.py", line 16

[tensor([[   37,  5551,  7706,    13, 32099,   344,     8,  1215, 18780,    15,
            11, 21485,  5718,     6,     3,  6443,  2035,     7,    15,   138,
            11,  6540,   760,     7,    15,   138,     6,    19,   230,  1702,
           294,    13, 21485,  5718,     5,     1]]), tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])]
[tensor([[21485,  5718,  4776,  4947,    28, 32099,     3,     5,     1]]), tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])]
[tensor([[32099, 26956,  5718, 32098,     1]]), tensor([[1, 1, 1, 1, 1]])]
[tensor([[32099,  7659, 32098,     1]]), tensor([[1, 1, 1, 1]])]
